# Instalação de bibliotecas

In [ ]:
!pip install tweepy
!pip install webdriver-manager

# Importação de bibliotecas

In [ ]:
import tweepy
import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup as bs

# Extraindo dados do Reclame Aqui

In [ ]:
def extrair_informacoes(base_url, empresa, driver, n_paginas):
    url = base_url + empresa + "/lista-reclamacoes/?pagina="
    reclamacoes, titulos, links = [], [], []

    for i in range(1, n_paginas + 1):
        pag = url + str(i)
        driver.get(pag)
        sleep(3)
        html = bs(driver.page_source, "html.parser")

        reclamacoes_html = html.find_all("p", {"class": 'text-detail'})
        reclamacoes_na_pagina = [
            reclamacao.text.split("|") for reclamacao in reclamacoes_html
        ]
        reclamacoes.extend(reclamacoes_na_pagina)

        titulos_e_links = html.find_all(
            "a", {"class": "link-complain-id-complains"}
        )

        titulos.extend([titulo.text.strip() for titulo in titulos_e_links])
        links.extend([link.get("href") for link in titulos_e_links])

    return reclamacoes, titulos, links


def extrair_descricoes(base_url, links, driver):
    urls = [base_url + link[1:] for link in links]
    descricoes = []
    notas = []
    for url in urls:
        driver.get(url)
        sleep(3)
        html = bs(driver.page_source, "html.parser")
        descricao_aux = html.find("div", {"class": "complain-body"})
        if descricao_aux is None:
            descricoes.append('Comentario Nulo')
        else:
            descricao = descricao_aux.text.strip()
            descricoes.append(descricao)

    return descricoes

In [ ]:
# driver = webdriver.Chrome('chromedriver')
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
base_url = 'https://www.reclameaqui.com.br/empresa/'
empresa = 'mercado-pago'

r, t, l = extrair_informacoes(base_url, empresa, driver, 3)

In [ ]:
%%time
d = extrair_descricoes(base_url, l, driver)

In [ ]:
len(d)

# Extraindo dados do Twitter

In [ ]:
api_key = '4Q60fmY53FcDWLPrmFLzMflqg'
api_secret_key = '8tJ8EKQyLE7hR2sUJ8RsFVpjvRoaHqWICwQfByFtwGHXTxFoic'

access_token = '1230518026015039488-Lof1DYgj97LOSViIzuh4hHgSrLudyf'
access_token_secret = 'BO8e5kIIrw5O4pX8GV2zkbZNwGZtLOmcZA9V4qMcc3X4K'

In [ ]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
text_query = '@MercadoPagoBr'
count = 200

try:
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.search, q=text_query).items(count)
 
    # Pulling information from tweets iterable object
    tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
    # Creation of dataframe from tweets list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list)

except BaseException as e:
    print('failed on_status,', str(e))
    time.sleep(3)

In [ ]:
tweets_df.columns = ['data', 'id', 'reclamacao']
tweets_df.shape

In [ ]:
tweets_df.head()

In [ ]:
tweets_df['titulo'] = 'Twitter'

# Salvando DataFrames

In [ ]:
reclamacoes_df = pd.DataFrame({'titulo': t, 'reclamacao': d})
reclamacoes_df.to_csv('base_reclame_aqui.csv', index=False)

In [ ]:
tweets_df[['titulo', 'reclamacao']].to_csv('base_twitter.csv', index=False)